In [1]:
import numpy as np
import  functools
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

In [2]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
df_data_交易=pd.read_csv('data/账户交易信息.csv')
df_data_交易.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']
df_data_交易 = df_data_交易.sort_values(["交易日期" , "交易时间"])
df_data_账户=pd.read_csv('data/账户静态信息.csv')
df_data_账户.columns = ['账户代号','开户日期','开户行代号','客户性别','年龄']
df_label_train=pd.read_csv('data/训练集标签.csv')
df_label_train.columns = ['账户代号','label']
df_label_test=pd.read_csv('data/test_dataset.csv')
df_label_test.columns = ['账户代号']## 读取数据
df_label_all=pd.concat([df_label_train,df_label_test])

In [4]:
list_交易_金额为负_index = list(df_data_交易[df_data_交易["交易金额"] < 0].index)
list_mz_index = [] ;
list_交易_金额为负_对应的失败交易_index = [] ;
for i in list_交易_金额为负_index :
    S_now = df_data_交易.loc[i] ;
    j = i ;
    while (True) :
        j -= 1 ;
        S_nex = df_data_交易.loc[j] ;
        if (S_now["交易金额"] == -S_nex["交易金额"]) :
            if (S_now["账户代号"] == S_nex["账户代号"]) :
                list_交易_金额为负_对应的失败交易_index.append(j) ;
                list_mz_index.append(i) ;
                break ;
list_交易_抹账需要去除_index = list_交易_金额为负_index + list_交易_金额为负_对应的失败交易_index ;
df_data_交易_去除抹账后=df_data_交易[~df_data_交易.index.isin(list_交易_抹账需要去除_index)]

In [5]:
df_data_交易_去除抹账后 = df_data_交易_去除抹账后.sort_values(["交易日期" , "交易时间"])

In [ ]:
# 查看余额波动
df_tmp = df_data_交易[df_data_交易["账户代号"] == "0187B03FED1591B0"].copy()
df_tmp.loc[: , "交易确认时间"] = df_tmp["交易日期"].to_numpy() + df_tmp["交易时间"].to_numpy()
plt.plot(df_tmp["交易确认时间"] , df_tmp["交易余额"])


In [ ]:
df_tmp

In [50]:
# 查找所有交易对手不是自己，又属于训练集或者测试集的交易
df_1 = df_data_交易_去除抹账后.copy()
df_2 = df_1[df_1["账户代号"] != df_1["对方账号"]]
l1 = df_label_train["账户代号"].to_numpy() ;
print(df_2[df_2["对方账号"].isin(l1)].shape)
# 结果：没有这样的交易

(0, 12)


In [9]:
#对于每个用户 查找是否有余额断层（即交易缺失）
gb1 = df_data_交易_去除抹账后.groupby("账户代号")

In [17]:
dict1 = defaultdict(int)
cnt = 2 ;
for name,df in gb1 :
    print (name)
    print (df.shape)
    yu_e = -1 ;
    
    for index, row in df.iterrows() :
        nex_yu_e = row["交易余额"] ;
        step = row["交易金额"]
        op = row["借贷标志"]
        if (op == 0) :
            step *= (-1) ;
        if (index != 1) :
            if (abs(yu_e + step - nex_yu_e) > 1e-5) :
                print (index)
                print (yu_e + step)
                print (nex_yu_e)
                print ("不正常") ;
        yu_e = nex_yu_e ;
    cnt-=1 ;
    if (cnt == 0) :
        break ;

00037295453A928A
(146, 12)
19251
2589.17
2652.09
不正常
212151
1577.38
1578.89
不正常
0004CC075464D54B
(37, 12)
3608
19999.0
22120.86
不正常
443253
25007.36
25011.14
不正常


In [18]:
df_tmp = df_data_交易[df_data_交易["账户代号"] == "0004CC075464D54B"]

In [19]:
df_tmp

,交易流水序号,账户代号,对方账号,借贷标志,交易金额,交易余额,对方行号,交易日期,交易时间,交易渠道,摘要代号,对方名称长度
3608,9385FDD7FC3D966BF8FF629413A998F2,0004CC075464D54B,D89699030A96FD18,1,20000.00,22120.86,2BE0C959,2020-03-01,12:02:27,621461AF,A3C65C29,6
3644,7FC4DCB43264C40D5B215BA6EFCA0D96,0004CC075464D54B,DB31FB6ACACC6844,0,22120.00,0.86,469F7281,2020-03-01,12:04:58,E96ED478,4E0CB6FB,24
7566,4458569DFAF8762FD32C7BB58EE0DC1B,0004CC075464D54B,96A1469E1E048CB6,1,25000.00,25000.86,7E78216E,2020-03-01,18:20:14,091D584F,2618045A,9
7592,B712DD148488D2A54E7A7457E9FC518D,0004CC075464D54B,A1E5D845B67D0DC6,0,25000.00,0.86,C89C9E84,2020-03-01,18:22:09,091D584F,A97DA629,6
12918,03E1B844CADD014FC43869606E3C9F1D,0004CC075464D54B,D89699030A96FD18,1,20000.00,20000.86,2BE0C959,2020-03-02,10:29:50,621461AF,A3C65C29,6
12998,8F2B41A638DF45371CAFBA4473B8B413,0004CC075464D54B,DB31FB6ACACC6844,0,20000.00,0.86,AACC16EA,2020-03-02,10:35:22,757B505C,0EFE3284,12
22221,85885797AC07EF75B85FA7BCEFF59247,0004CC075464D54B,DB31FB6ACACC6844,0,0.50,0.36,469F7281,2020-03-03,10:02:02,E96ED478,4E0CB6FB,24
25133,3C272EC05FF5EAAD1E5CB2E91D5EC677,0004CC075464D54B,15BD84FDE48CB422,1,5000.00,5000.36,F5CD2AD7,2020-03-03,16:25:47,621461AF,A3C65C29,9
25142,3659E1FDFDD22515DF301831314850BE,0004CC075464D54B,D0DB315FE0FF5625,0,4882.00,118.36,8A1BC467,2020-03-03,16:28:02,091D584F,A97DA629,9
25144,EECC212D0880E5E0563DC62A49DBF1B9,0004CC075464D54B,4424DB4F1CE6F25D,1,50000.00,50118.36,583BDDDA,2020-03-03,16:28:14,621461AF,A3C65C29,6


In [75]:
# 获取正常交易转出方 次数最多
df_1 = df_data_交易.copy()
l1 = df_label_train["账户代号"].to_numpy() ;
df_2 = df_1[df_1["账户代号"].isin(l1)]
df_2 = df_2.merge(df_label_train, on="账户代号" , how="left")

df_3 = df_2[df_2["label"] == 1]
df_4 = df_3.groupby("对方账号")[["账户代号" , "交易金额"]].agg({"账户代号" : ["nunique"] , "交易金额" :["mean" , "sum" , "count"]}).reset_index()
ll = [] ;
for i in df_4.columns :
    now = i[0] + '_1_' +  i[1] ;
    if i[1] == "" :
        now = i[0] ;
    ll.append(now) ;
df_data_转出方_1 = df_4.copy() ;
df_data_转出方_1.columns = ll ;



df_3 = df_2[df_2["label"] == 0]
df_4 = df_3.groupby("对方账号")[["账户代号" , "交易金额"]].agg({"账户代号" : ["nunique"] , "交易金额" :["mean" , "sum" , "count"]}).reset_index()
ll = [] ;
for i in df_4.columns :
    now = i[0] + '_0_' +  i[1] ;
    if i[1] == "" :
        now = i[0] ;
    ll.append(now) ;
df_data_转出方_0 = df_4.copy() ;
df_data_转出方_0.columns = ll ;

In [82]:
df_data_转出方_0.sort_values("账户代号_0_nunique" , ascending=False).head(10)

,对方账号,账户代号_0_nunique,交易金额_0_mean,交易金额_0_sum,交易金额_0_count
784,0F76A363EACBDEAF,644,2464.72,44633658.93,18109
10970,EDE31EF0F69DC342,591,1836.18,15480812.08,8431
1059,14BEFED1370B730A,560,740.61,12491067.48,16866
1540,1F77C9FDBDBE78D0,500,191.41,1383306.35,7227
10231,DE7ABC0638690944,500,197.69,1411868.47,7142
10026,D98BD9FA9D006723,344,3077.56,8675639.02,2819
10096,DB31FB6ACACC6844,263,1312.55,2538479.35,1934
8630,BC2042D38BEE2B23,258,193.02,274470.67,1422
5385,731CB06E4FF912A2,214,51.05,74732.44,1464
3620,4D17889E455921CA,213,52.69,77192.41,1465


In [84]:
df_data_转出方_1.sort_values("账户代号_1_nunique" , ascending=False).head(10)

,对方账号,账户代号_1_nunique,交易金额_1_mean,交易金额_1_sum,交易金额_1_count
882,0F76A363EACBDEAF,254,4045.36,12168447.36,3008
12454,D98BD9FA9D006723,243,903.94,1800650.74,1992
12571,DB31FB6ACACC6844,228,561.85,904574.52,1610
1201,14BEFED1370B730A,197,817.90,3475277.91,4249
13589,EDE31EF0F69DC342,148,1234.86,2130141.07,1725
10662,BA8C172953D66632,145,770.80,369984.46,480
12759,DE7ABC0638690944,131,289.37,393258.85,1359
1832,1F77C9FDBDBE78D0,124,283.88,382671.60,1348
6681,7495D485C0213E9C,66,256.85,50342.43,196
10757,BC2042D38BEE2B23,66,633.31,274857.19,434


In [107]:
df_data_转出方 = df_data_转出方_1.merge(df_data_转出方_0 , on="对方账号" , how="left")
df_data_转出方 = df_data_转出方.replace(np.nan, 0)

In [109]:
df_data_转出方["账户代号_出现次数差值"] = df_data_转出方["账户代号_1_nunique"].to_numpy() - df_data_转出方["账户代号_0_nunique"].to_numpy()

In [114]:
df_data_转出方.sort_values("账户代号_出现次数差值" , ascending=False).head(30)

,对方账号,账户代号_1_nunique,交易金额_1_mean,交易金额_1_sum,交易金额_1_count,账户代号_0_nunique,交易金额_0_mean,交易金额_0_sum,交易金额_0_count,账户代号_出现次数差值
8786,98DFCBF3A3C16C9B,39,49.27,4828.17,98,12.00,61.11,1222.30,20.00,27.00
14409,FAEFBBD20A63E0A1,20,13.89,1221.90,88,8.00,13.37,254.10,19.00,12.00
2565,2BF17C5F2AC8595F,7,6544.78,111261.31,17,1.00,500.00,500.00,1.00,6.00
10887,BE51079F1C0B5864,9,606.47,11523.00,19,3.00,756.56,3026.25,4.00,6.00
11103,C21B6C56D13C96E3,5,77.43,1781.00,23,0.00,0.00,0.00,0.00,5.00
10503,B81E01214EA28BC9,6,176.91,1946.00,11,1.00,100.00,200.00,2.00,5.00
13759,F08678AA579A9158,9,14.55,305.61,21,4.00,25.56,127.80,5.00,5.00
9618,A815057880A279D4,4,12.25,73.50,6,0.00,0.00,0.00,0.00,4.00
4942,562191D789884974,4,58059.65,3657758.00,63,0.00,0.00,0.00,0.00,4.00
4695,51A834AD302FB96F,5,416.91,3335.24,8,1.00,35035.00,35035.00,1.00,4.00


In [116]:
df_data_转出方[df_data_转出方["账户代号_出现次数差值"] >= 3]["对方账号"].to_numpy()

array(['0835BD730DA89A44', '2BF17C5F2AC8595F', '332B93A07873A8F4',
       '3FC6809541A40E95', '51A834AD302FB96F', '562191D789884974',
       '8F2CD8B7066B256E', '98DFCBF3A3C16C9B', 'A815057880A279D4',
       'B81E01214EA28BC9', 'BE51079F1C0B5864', 'C21B6C56D13C96E3',
       'E271F81AD7E2CBA7', 'EF8CBFA2E6C4DB8B', 'F08678AA579A9158',
       'F145BA87DFB249F2', 'FAEFBBD20A63E0A1', 'FBB09C45DFB71FAD',
       'FEE39FD6690BC42C'], dtype=object)